# Things i'll need

In [21]:
import nltk
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim 

# Importing csv file

In [11]:
tweets_csv = pd.read_csv('tweets.csv')
docs = list(tweets_csv['text'])

# Preprocessing

In [17]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = str(docs[idx]).lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [18]:
docs = docs_preprocessor(docs)

In [19]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/Users/chris/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


# Creating a Corpus

In [22]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 230440
Number of unique words after removing rare and common words: 19774


In [24]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 19774
Number of documents: 203451


# Creating LDA Model

In [25]:
from gensim.models import LdaModel
num_topics = 20
chunksize = 500
passes = 15
iterations = 100
eval_every = 1

temp = dictionary[0]  # This is only to \"load\" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

/Users/chris/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 57min 12s, sys: 40 s, total: 57min 52s
Wall time: 2h 35min 8s


In [26]:
model.save('lda.model')

# Displaying Model

In [27]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [28]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

/Users/chris/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.019794  0.349719       1        1  11.323096
1      0.039046 -0.017167       2        1   6.732718
0      0.041103 -0.036296       3        1   5.688047
15     0.054374 -0.092132       4        1   5.635257
5      0.037182 -0.020455       5        1   5.191975
17     0.037562 -0.019789       6        1   5.121435
2      0.040789 -0.025827       7        1   5.096361
18     0.037661 -0.019517       8        1   5.030825
14     0.047868 -0.066318       9        1   4.856785
16     0.037296 -0.020099      10        1   4.690160
6     -0.331616 -0.059758      11        1   4.535310
7      0.036628 -0.020920      12        1   4.329830
9      0.049373 -0.069368      13        1   4.258439
19     0.036178 -0.021356      14        1   4.116919
3      0.039542 -0.026767      15        1   4.089090
11     0.036205 -0.021331      16        1   4.024917
8     -0.331201 -0.012422      17        1   4.010527
13     0.047336 -0.073200      18        1   3.960440
10     0.035904 -0.021614      19        1   3.835068
4      0.028564  0.294615      20        1   3.472795, topic_info=     Category          Freq                   Term         Total  loglift  \
term                                                                        
40    Default  31942.000000                  trump  31942.000000  30.0000   
23    Default  13136.000000                clinton  13136.000000  29.0000   
169   Default  13898.000000           donald_trump  13898.000000  28.0000   
25    Default  11684.000000        hillary_clinton  11684.000000  27.0000   
45    Default  15854.000000                   that  15854.000000  26.0000   
24    Default  11355.000000                hillary  11355.000000  25.0000   
76    Default  14905.000000                   this  14905.000000  24.0000   
81    Default  10650.000000                  obama  10650.000000  23.0000   
155   Default  13085.000000                   with  13085.000000  22.0000   
68    Default   9057.000000                   your   9057.000000  21.0000   
122   Default   7751.000000                  about   7751.000000  20.0000   
186   Default   7166.000000                   just   7166.000000  19.0000   
107   Default   9405.000000                   will   9405.000000  18.0000   
336   Default   8848.000000                   they   8848.000000  17.0000   
59    Default   8563.000000                   what   8563.000000  16.0000   
177   Default   8548.000000                   have   8548.000000  15.0000   
11    Default   6159.000000                   like   6159.000000  14.0000   
15    Default   5924.000000                   when   5924.000000  13.0000   
220   Default   7836.000000                   from   7836.000000  12.0000   
4     Default   5710.000000                   more   5710.000000  11.0000   
35    Default   7641.000000                 people   7641.000000  10.0000   
771   Default   5486.000000        realdonaldtrump   5486.000000   9.0000   
281   Default   4326.000000                america   4326.000000   8.0000   
193   Default   4759.000000                   make   4759.000000   7.0000   
168   Default   5026.000000                 donald   5026.000000   6.0000   
170   Default   4949.000000                   news   4949.000000   5.0000   
217   Default   4408.000000                   time   4408.000000   4.0000   
435   Default   4058.000000                   tcot   4058.000000   3.0000   
188   Default   4258.000000               american   4258.000000   2.0000   
162   Default   3894.000000                   vote   3894.000000   1.0000   
...       ...           ...                    ...           ...      ...   
787   Topic20   3029.844727                   here   3030.006592   3.3602   
180   Topic20   2515.095459        trump_supporter   2515.256836   3.3601   
179   Topic20   1611.000732              supporter   1611.161621   3.3601 